In [2]:
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image
import torch

In [3]:
path = 'images/'
birds=['120.Fox_Sparrow','080.Green_Kingfisher','187.American_Three_toed_Woodpecker',
       '127.Savannah_Sparrow','043.Yellow_bellied_Flycatcher','041.Scissor_tailed_Flycatcher','114.Black_throated_Sparrow']

In [ ]:
X=[]
Y=[]
for (bird,i) in  zip(birds,np.arange(7)):
    for file in os.listdir(path+bird):
        #print(file)
        data=Image.open(path + bird + '/' + file, 'r')
        width, height = data.size  
        left = width/10
        top = height / 10
        right = 9*width/10
        bottom = 9* height / 10
        data = data.crop((left, top, right, bottom))
        data=data.resize((50,50))
        data=np.array(data).astype('float32')
        data=data/255.0
        data = [torch.tensor(data[:,:,0]),torch.tensor(data[:,:,1]),torch.tensor(data[:,:,2])]
        data = torch.stack(data,0)
        X.append(data)
        Y.append(torch.tensor(i))

In [ ]:
torch.save(torch.stack(X),'X50.pt')
torch.save(torch.stack(Y),'Y50.pt')
torch.save(torch.randperm(len(Y)),'P.pt')

In [4]:
X = torch.load('X50.pt')
Y = torch.load('Y50.pt')
p = torch.load('P.pt')

In [5]:
X_tr=X[p[:287]]
Y_tr=Y[p[:287]]

X_v=X[p[287:328]]
Y_v=Y[p[287:328]]

X_te=X[p[328:]]
Y_te=Y[p[328:]]

In [6]:
train_dataset = torch.utils.data.TensorDataset(X_tr,Y_tr)
validation_dataset = torch.utils.data.TensorDataset(X_v,Y_v)
test_dataset = torch.utils.data.TensorDataset(X_te,Y_te)

In [ ]:
batch_size = 16
tr= torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,shuffle=True,num_workers=8)
v= torch.utils.data.DataLoader(validation_dataset, batch_size=batch_size,shuffle=True,num_workers=8)
te= torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,shuffle=True,num_workers=8)

In [8]:
np.shape(X_tr[0])

torch.Size([3, 50, 50])

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.cl1 = torch.nn.Conv2d(3,4,3,stride=1,padding=1)
        self.mp1 = torch.nn.AvgPool2d(2,ceil_mode=True)
        self.cl2 = torch.nn.Conv2d(4,16,3,stride=1,padding=1)
        self.mp2 = torch.nn.AvgPool2d(2,ceil_mode=True)
        self.drop=torch.nn.Dropout(0.25)
        self.fc1 = torch.nn.Linear(13*13*16,21) # 25*25*16 for 100, 13*13*16 for 50
        self.fc2 = torch.nn.Linear(21,7)

    def forward(self,inp):
        inp = self.cl1(inp)
        inp = self.mp1(inp)
        inp = self.cl2(inp)
        inp = self.mp2(inp)
        inp = torch.flatten(inp, 1)
        inp = self.drop(inp)
        inp = torch.nn.functional.relu(self.fc1(inp),inplace=True)
        inp = self.fc2(inp)
        return inp

In [ ]:
net = Net()
c_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr = 0.0001)
# optimizer = torch.optim.SGD(net.parameters(), lr=0.005, momentum=0.0,nesterov=False)
tr_loss=[]
tr_corrects=[]
v_loss=[]
v_corrects=[]

In [ ]:
for count in range(1):
    l=0;c=0;l1=0;c1=0
    for inp,out in tr:
        results = net(inp)
        out=out.type(torch.LongTensor)
        loss=c_fn(results,out)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        temp,predictions=torch.max(results, 1)
        l+=loss.item()
        c+=torch.sum(predictions==out.data)


    for inp, out in v:
        results = net(inp)
        out=out.type(torch.LongTensor)
        loss=c_fn(results,out)
        temp,predictions=torch.max(results, 1)
        l1+=loss.item()
        c1+=torch.sum(predictions==out.data)

    if ((count+1)%20==0):
        tot_loss = l/len(tr.dataset)
        tot_acc = c.float()/len(tr.dataset)
        v_tot_loss = l1/len(v.dataset)
        v_tot_acc = c1.float()/len(v.dataset)
        tr_loss.append(tot_loss)
        tr_corrects.append(tot_acc)
        v_loss.append(v_tot_loss)
        v_corrects.append(v_tot_acc)
        print("after ",(count+1),"iterations :training loss is ",tot_loss," and validation loss is ",v_tot_loss,",training accuracy is "
              ,int(tot_acc.item()*100),",validation accuracy is ",int(v_tot_acc.item()*100))
    

In [ ]:
c=0;
for inp,out in te:
    results = net(inp)
    out=out.type(torch.LongTensor)
    loss=c_fn(results,out)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    temp,predictions=torch.max(results, 1)
    c+=torch.sum(predictions==out.data)

test_acc =c.float()/len(te.dataset)
print(test_acc)

In [ ]:
plt.plot(tr_loss, label='training loss')
plt.plot(v_loss, label='validation loss')
plt.legend()
plt.show()
plt.plot(tr_corrects, label='training accuracy')
plt.plot(v_corrects, label='validation accuracy')
plt.legend()
plt.show()

In [ ]:
class Net1(torch.nn.Module):
    def __init__(self,alpha,k):
        super().__init__()
        self.cl1 = torch.nn.Conv2d(3,4,3,stride=1,padding=1)
        self.mp1 = torch.nn.AvgPool2d(2,ceil_mode=True)
        self.cl2 = torch.nn.Conv2d(4,16,3,stride=1,padding=1)
        self.mp2 = torch.nn.AvgPool2d(2,ceil_mode=True)
        self.alpha = alpha
        self.k = k
        self.NETVLAD_Conv = torch.nn.Conv2d(16, self.k, kernel_size=(1, 1), bias=True)
        self.centroids = torch.nn.Parameter(torch.rand(self.k, 16))
        self.NETVLAD_Conv.weight = torch.nn.Parameter((2.0 * self.alpha * self.centroids).unsqueeze(-1).unsqueeze(-1))
        self.NETVLAD_Conv.bias = torch.nn.Parameter(- self.alpha * self.centroids.norm(dim=1))
        self.drop=torch.nn.Dropout(0.25)
        self.fc1 = torch.nn.Linear(16*self.k,21)
        self.fc2 = torch.nn.Linear(21,7)

    def forward(self,inp):
        inp = self.cl1(inp)
        inp = self.mp1(inp)
        inp = self.cl2(inp)
        inp = self.mp2(inp)

        N, C = inp.shape[:2]
        inp = torch.nn.functional.normalize(inp, p=2, dim=1)
        soft = self.NETVLAD_Conv(inp).view(N, self.k, -1)
        soft = torch.nn.functional.softmax(soft, dim=1)
        inpflat = inp.view(N, C, -1)
        res = inpflat.expand(self.k, -1, -1, -1).permute(1, 0, 2, 3) - self.centroids.expand(inpflat.size(-1), -1, -1).permute(1, 2, 0).unsqueeze(0)
        res *= soft.unsqueeze(2)
        netvlad = res.sum(dim=-1)
        netvlad = torch.nn.functional.normalize(netvlad, p=2, dim=2)
        netvlad = netvlad.view(inp.size(0), -1)
        inp = torch.nn.functional.normalize(netvlad, p=2, dim=1)

        inp = self.drop(inp)
        inp = torch.nn.functional.relu(self.fc1(inp),inplace=True)
        inp = self.fc2(inp)
        return inp

In [ ]:
net = Net1(0.5,4)
c_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr = 0.0005)
tr_loss=[]
tr_corrects=[]
v_loss=[]
v_corrects=[]

In [ ]:
for count in range(200):
    l=0;c=0;l1=0;c1=0
    for inp,out in tr:
        results = net(inp)
        out=out.type(torch.LongTensor)
        loss=c_fn(results,out)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        temp,predictions=torch.max(results, 1)
        l+=loss.item()
        c+=torch.sum(predictions==out.data)


    for inp, out in v:
        results = net(inp)
        out=out.type(torch.LongTensor)
        loss=c_fn(results,out)
        temp,predictions=torch.max(results, 1)
        l1+=loss.item()
        c1+=torch.sum(predictions==out.data)

    if ((count+1)%20==0):
        tot_loss = l/len(tr.dataset)
        tot_acc = c.float()/len(tr.dataset)
        v_tot_loss = l1/len(v.dataset)
        v_tot_acc = c1.float()/len(v.dataset)
        tr_loss.append(tot_loss)
        tr_corrects.append(tot_acc)
        v_loss.append(v_tot_loss)
        v_corrects.append(v_tot_acc)
        print("after ",(count+1),"iterations :training loss is ",tot_loss," and validation loss is ",v_tot_loss,",training accuracy is "
              ,int(tot_acc.item()*100),",validation accuracy is ",int(v_tot_acc.item()*100))
    

In [ ]:
c=0
for inp,out in te:
    results = net(inp)
    out=out.type(torch.LongTensor)
    loss=c_fn(results,out)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    temp,predictions=torch.max(results, 1)
    c+=torch.sum(predictions==out.data)

test_acc =c.float()/len(te.dataset)
print(test_acc)

In [ ]:
plt.plot(tr_loss, label='training loss')
plt.plot(v_loss, label='validation loss')
plt.legend()
plt.show()
plt.plot(tr_corrects, label='training accuracy')
plt.plot(v_corrects, label='validation accuracy')
plt.legend()
plt.show()